# BioImage Timelapse Processor: Multi-Dimensional ND2 to MP4
## Automated Visualization Pipeline for Live-Cell Imaging Data (XYZCTS)

### Read Me
This notebook automates the processing of raw Nikon ND2 microscopy files into presentation-ready video files. It is designed to handle multi-dimensional data containing Time (T), Z-stacks (Z), Multi-positions (S), and Channels (C).
#### Key Features
- Metadata Extraction: Automatically reads physical dimensions (x, y, z) and time intervals (t).
- Z-Projection: Converts 3D stacks into 2D images using Maximum Intensity Projection (MIP).
- Auto-Normalization: Applies percentile-based normalization to enhance contrast while ignoring hot pixels.
- Custom Visualization: Applies user-defined colormaps (LUTs) to specific channels.
- Automatic Annotation:
    - Scale Bar: Draws a physical scale bar based on metadata pixel size.
    - Timestamp: Calculates and stamps "HH:MM" on every frame based on acquisition intervals.
- Export: Saves individual channels and merged overlays as .mp4 video files.
- Batch Processing: Can process a single file or iterate through an entire directory.

#### How to Use
1. Input Data: Place your .nd2 files in a local directory.
2. Configuration (Section 3): Update the WorkingDir path to your folder.
3. Channel Mapping: Update the ChannelInfo dictionary to match your specific experiment (e.g., Channel 0 = "DAPI", Color = "Blue").
4. Run: Execute all cells. The script will create a _Processed folder containing the output videos.

### 1. Importing Libraries & Defining Functions


In [ ]:
# Importing necessary libraries
from pathlib import Path
import numpy as np
import math
from PIL import Image, ImageDraw, ImageFont
from matplotlib.colors import LinearSegmentedColormap
from bioio import BioImage
import bioio_nd2
from bioio_imageio.writers import TimeseriesWriter
import shutil

# -----------------------------------------------------------------------------
# 2.1 FILE HANDLING FUNCTIONS
# -----------------------------------------------------------------------------

# Function to create output directory
# Logic: Takes the parent folder name, appends a suffix, creates the folder if it doesn't exist.
def MakeDir(WorkingDir: Path, Suffix: str):
    FolderName = WorkingDir.name
    outdir = WorkingDir / f"{FolderName}_{Suffix}"
    outdir.mkdir(exist_ok=True) # exist_ok=True prevents errors if folder already exists
    print(f"Output directory: {outdir}")
    return outdir

# Function to create sub-directory
# Logic: Creates a sub-directory within a given parent directory.
def MakeSubDir(ParentDir: Path, SubDirName: str):
    subdir = ParentDir / SubDirName
    subdir.mkdir(exist_ok=True) # exist_ok=True prevents errors if folder already exists
    print(f"Sub-directory created: {subdir}")
    return subdir

# Function to remove file extensions
# Logic: Iterates through a known list of microscopy extensions and strips them to get the base filename.
def rm_ext(FileName):
    extensions = [".ome.tiff", ".ome.tif", ".tiff", ".tif", ".nd2", ".czi", ".lif", ".oir"]
    for ext in extensions:
        if FileName.endswith(ext):
            FileName = FileName.replace(ext, "")
            break
    return FileName

# Function to make the file list to process
# Logic: Handles "DryRun" (testing) and standard batch processing.
def FileList(
    WorkingDir: Path,
    FileToProcess: str,
    DryRun: bool
):
    # Dry Run: First image only
    # Useful for testing parameters on a single file before running a large batch.
    if DryRun == True: 
        ND2_File = next(WorkingDir.glob("*.nd2"), None) # Next function to get the first & glob to find files matching pattern
        if ND2_File is None:
            print("No ND2 files found")
            return []
        
        print("Dry run enabled. Only processing the first ND2 file:", ND2_File.name)
        return [ND2_File]
    # End of DryRun if block
    
    # Normal Run: Find all ND2 files
    ND2_File = list(WorkingDir.glob("*.nd2"))
    # Error handling if no ND2 files found
    if not ND2_File: 
        print("No ND2 files found")
        return []

    # Process all ND2 files
    if FileToProcess == "all":
        print(f"Processing all ND2 files in the directory. Total files found: {len(ND2_File)}")
        return ND2_File
    # Process a specific ND2 file by name
    for file in ND2_File:
        if file.name == FileToProcess:
            print(f"Processing specific ND2 file: {FileToProcess}")
            return [file]
    print(f"Requested file not found: {FileToProcess}")
    return []

# -----------------------------------------------------------------------------
# 2.2 ND2 PROCESSING FUNCTIONS
# -----------------------------------------------------------------------------

# Function to extract ND2 metadata
# Logic: Uses bioio property accessors to get physical dimensions (microns) and time intervals.
def ND2_Metadata(img):

    std_md = img.standard_metadata
    # Checking the timelapse metadata existence
    if getattr(std_md, "timelapse_interval", None) is not None:
        delta_time = round(std_md.timelapse_interval.total_seconds() / 60) * 60  # Rounded to nearest minute (in seconds)
    else:
        delta_time = None

    if getattr(std_md, "total_time_duration", None) is not None:
        duration_secs = std_md.total_time_duration.total_seconds()
        duration_hrs = round(duration_secs / 3600)
    else:
        duration_secs = None
        duration_hrs = None

    n_frames = std_md.image_size_t
    if n_frames is None or n_frames < 1: n_frames = 1
    
    # Extracting metadata into a clean dictionary
    md = {
        "DimOrder": std_md.dimensions_present,      # e.g., 'TCZYX'
        "nScenes": len(img.scenes),                 # Number of XY positions
        "nFrames": n_frames,             # Timepoints
        "nChannels": std_md.image_size_c,           # Channels
        "nSlices": std_md.image_size_z,             # Z-planes
        "nPixelsY": std_md.image_size_y,            # Height
        "nPixelsX": std_md.image_size_x,            # Width
        "xPixel_size_um": std_md.pixel_size_x,      # Calibration X
        "yPixel_size_um": std_md.pixel_size_y,      # Calibration Y
        "zPixel_size_um": std_md.pixel_size_z,      # Calibration Z
        "deltaTime": delta_time,
        "Duration_Secs": duration_secs,
        "Duration_Hrs": duration_hrs
    }

    # Printing metadata for user verification
    print(
        f"Dimension Order: {md['DimOrder']}, "
        f"Scenes: {md['nScenes']}, "
        f"Frames: {md['nFrames']}, "
        f"Channels: {md['nChannels']}, "
        f"Slices(Z): {md['nSlices']}, "
        f"Pixels(Y,X): ({md['nPixelsY']},{md['nPixelsX']})"
    )
    print(
        f"Pixel Size: X={md['xPixel_size_um']} µm, "
        f"Y={md['yPixel_size_um']} µm, "
        f"Z={md['zPixel_size_um']} µm"
    )
    print(
        f"Frame Interval: {md['deltaTime']} s, "
        f"Total Duration: {md['Duration_Secs']} s, "
        f"Total Duration: {md['Duration_Hrs']} Hrs"
    )

    return md

# Function to make the scene list to process
# Logic: Decides whether to process all scenes (XY positions) or a user-defined subset.
def SceneList(
    img,
    ScenesToProcess: str | list[int],
    DryRun: bool
):
    nScenes = len(img.scenes)

    # Dry Run: First image only (Scene 0)
    if DryRun == True: 
        print("Dry run enabled. Only processing the first scene: 0")
        return [0]
    # End of DryRun if block
    
    # Normal Run
    AllScenes = list(range(nScenes))
    # Error handling if no scenes found
    if nScenes == 0: 
        print("No scenes found")
        return []
    # Process all scenes in the file
    if ScenesToProcess == "all":
        print(f"Processing all scenes in the ND2 file. Total scenes found: {nScenes}")
        return AllScenes
    
    # Process a specific list of scenes provided by user
    if isinstance(ScenesToProcess, list):

        # Optional: validate indices to ensure they exist in the file
        valid_scenes = [s for s in ScenesToProcess if 0 <= s < nScenes]
        if not valid_scenes:
            print("No valid scenes specified")
            return []
        print(f"Processing specific scenes: {valid_scenes}")
        return valid_scenes
    print("Invalid value for ScenesToProcess")
    return []

# -----------------------------------------------------------------------------
# 2.3 VISUALIZATION FUNCTIONS
# -----------------------------------------------------------------------------
# Function to get channel data with correct dimensionality
# Logic: Depending on presence of T and Z axes, retrieves image data in appropriate shape.
def get_channel_data(img, MD, channel_idx):
    nFrames = int(MD.get("nFrames") or 1)
    nSlices = int(MD.get("nSlices") or 1)

    if nFrames > 1 and nSlices > 1: order = "TZYX"
    elif nFrames == 1 and nSlices > 1: order = "ZYX"
    elif nFrames > 1 and nSlices == 1: order = "TYX"
    else: order = "YX"

    cimg = img.get_image_data(order, C=channel_idx)
    return cimg, order

# Function to determine if the dataset is a timelapse
# Logic: Checks for presence of Time axis, multiple frames, and non-zero deltaTime.
def IsTimelapse(MD, cimg_dimorder):
    nFrames = MD.get("nFrames")
    deltaTime = MD.get("deltaTime")
    has_Time_axis = 'T' in cimg_dimorder
    has_nframes = nFrames is not None and nFrames > 1
    has_deltaTime = deltaTime is not None and deltaTime > 0
    return True if has_Time_axis and has_nframes and has_deltaTime else False

# Defining projection function
# Logic: Projects a 3D/4D stack along a specified axis using min, max, sum, mean, or standard deviation.
def project(
    stack: np.ndarray,
    mode: str,
    axis: str,
    dim_order: str,
    keepdims: bool
):
    if isinstance(axis, str): axis_idx = dim_order.index(axis) # Convert axis letter to index
    
    # Use float for safe arithmetic where needed
    mode = mode.lower()
    if mode in {"sum", "mean", "sd"}: data = stack.astype(np.float32)
    else: data = stack

    print(f"Projecting {axis} axis, which is axis index {axis_idx}/{dim_order} in {mode} mode.")

    # Perform projection based on mode
    if mode == "min": return np.min(data, axis=axis_idx, keepdims=keepdims)
    elif mode == "max": return np.max(data, axis=axis_idx, keepdims=keepdims)
    elif mode == "sum": return np.sum(data, axis=axis_idx, keepdims=keepdims)
    elif mode == "mean": return np.mean(data, axis=axis_idx, keepdims=keepdims)
    elif mode == "sd": return np.std(data, axis=axis_idx, keepdims=keepdims)
    else: raise ValueError( "Invalid mode. Choose one of: 'min', 'max', 'sum', 'mean', 'sd'")

# Defining normalization function (global or percentile based)
# Logic: Normalizes image intensity to 0-1 range. Percentile helps ignore hot pixels/outliers.
def norm_img(img, percent_norm=True, lower_pct=0.35, upper_pct=99.65): #percent_norm=True for percentile normalization
    nFrame = img.shape[0]
    mid_frame = nFrame // 2           # index for middle frame
    ref_frame = img[mid_frame]   # reference frame used to calculate stats (assumed representative)

    flat = ref_frame.flatten()
    if percent_norm:
        low = np.percentile(flat, lower_pct)
        high = np.percentile(flat, upper_pct)
    else:
        low = flat.min()
        high = flat.max()

    denom = high - low if high != low else 1
    normed = (img - low) / denom
    normed_img = np.clip(normed, 0, 1) # Clips values outside the low/high range

    print(
    f"Normalizing image using {'percentile, clipping ± ' + f'{100 - upper_pct:.2f}% values' if percent_norm else 'min–max'} method.") 
    return normed_img

# Defining colormaps
# Logic: Generates a Matplotlib LinearSegmentedColormap (e.g., Black -> Green) and applies it to the image.
def cmap(img, LUT: str, Isinverted: bool):
    BaseColors = { # Dictionary of base colors
        "red":     (1, 0, 0),
        "green":   (0, 1, 0),
        "blue":    (0, 0, 1),
        "yellow":  (1, 1, 0),
        "magenta": (1, 0, 1),
        "cyan":    (0, 1, 1),
        "gray":    (1, 1, 1),
    }
    assert LUT in BaseColors, f"Invalid LUT name: {LUT}"
    color = BaseColors[LUT] # Calling the base color value
    if Isinverted:
        # White background to Color
        img_cmap = LinearSegmentedColormap.from_list(f"{LUT}_inv", [color, (1, 1, 1)], N=256)
    else:
        # Black background to Color
         img_cmap = LinearSegmentedColormap.from_list(LUT, [(0, 0, 0), color], N=256)
    
    # Apply LUT → RGB → uint8 conversion for standard video compatibility
    img_rgb = img_cmap(img)[..., :3]
    img_rgb_8bit = (img_rgb * 255).astype(np.uint8)
    print(f"Applying colormap: {LUT}")
    return img_rgb_8bit

# Defining font size function
# Logic: Calculates font size dynamically based on 6% of the image height.
def font_size(img, scaling=0.06, min_size=10, max_size=48):
    height = img.shape[0] # height of the image
    font_size = int(height * scaling) # 6% of image height
    font_size = max(min_size, min(max_size, font_size)) # Clamp between min and max
    return font_size

# Defining scale bar function
# Logic: Draws a physical scale bar (e.g., 10um) on the image using PIL.
def scale_bar(img, xPixel_size_um, color=(255, 255, 255), label=True):
        
    h, w = img.shape[:2] # height and width
    max_len = w * xPixel_size_um * 0.2  # Target length approx 20% of image width
    
    # Picking suitable engineering series (1, 2, 5) to get a round number (e.g. 10, 20, 50 um)
    exponent = int(math.floor(math.log10(max_len)))
    candidates = [s * 10**exponent for s in [5, 2, 1]]
    scale_bar_len_um = next((c for c in candidates if c <= max_len), max_len)

    # Calculating scale bar dimensions in Pixels
    scale_bar_px_length = int(round(scale_bar_len_um / xPixel_size_um)) # scale bar length in pixels
    scale_bar_px_height = max(int(h * 0.01), 2) # scale bar height in pixels (1% of height)
    
    # Positioning scale bar at bottom-left corner
    bar_x = w - int(w * 0.05) # 5% from side
    bar_y = h - int(h * 0.035) # 3.5% from bottom

    # Calculating bar coordinates for lower right
    bar_x_end = bar_x
    bar_x_start = bar_x_end - scale_bar_px_length

    # Convert NumPy array to PIL Image for drawing
    pil_img = Image.fromarray(img)

    # Draw rectangle (scale bar) with PIL
    draw = ImageDraw.Draw(pil_img)
    draw.rectangle(
        [(bar_x_start, bar_y), (bar_x_start + scale_bar_px_length, bar_y + scale_bar_px_height)],
        fill=color)

    # Convert back to NumPy array if needed
    img_scaled = np.array(pil_img)

    # Draw scale bar label (with PIL)
    font_sz = font_size(img_scaled) # Calling the font size function
    if label:
        # Convert numpy image array to PIL Image
        pil_img = Image.fromarray(img_scaled)
        draw = ImageDraw.Draw(pil_img)
        # Use a truetype font if available, or fallback to the default font
        try:
            font = ImageFont.truetype("arial.ttf", font_sz)
        except Exception:
            font = ImageFont.load_default()
        label_text = f"{scale_bar_len_um:g} µm"  # Unicode micro symbol
        
        # Centering text above the scale bar
        bbox = draw.textbbox((0,0), label_text, font=font)  # Retriving the bounding box of the text
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        
        # Calculating text position
        bar_center = bar_x_start + ((bar_x_end - bar_x_start) // 2)
        text_x = bar_center - (text_width // 2)
        text_y = bar_y - 8 - text_height     # 8 pixel gap above scale bar

        # Use a truetype font if available, or fallback to the default font
        try:
            font = ImageFont.truetype("arial.ttf", font_sz)
        except Exception:
            font = ImageFont.load_default()
        
        # Add the text to the image
        draw.text((text_x, text_y), label_text, font=font, fill=color)
        img_scaled = np.array(pil_img)

    return img_scaled, scale_bar_len_um

# Defining timestamp function
# Logic: Calculates 'Hours:Minutes' based on frame index and deltaTime, draws text on Top-Left.
def timestamp(img, frame_idx, deltaTime, color=(255, 255, 255)):
    
    font_sz = font_size(img) # Calling the font size function
    # Convert numpy (OpenCV) image array to PIL Image
    pil_img = Image.fromarray(img)
    draw = ImageDraw.Draw(pil_img)
    # Use a truetype font if available, or fallback to the default font
    try:
        font = ImageFont.truetype("arial.ttf", font_sz)
    except Exception:
        font = ImageFont.load_default()
    
    # Calculate timestamp for this frame
    Duration = frame_idx * deltaTime
    hours = int(Duration // 3600)
    minutes = int((Duration % 3600) // 60)
    timestamp_str = f"{hours:02d}:{minutes:02d} Hrs"
    
    text_x, text_y = 10, 10 # Top left corner, with a small padding
    draw.text((text_x, text_y), timestamp_str, font=font, fill=color)
    img_timed = np.array(pil_img)
    return img_timed

# Defining annotation function for timelapse
# Logic: Wrapper that iterates through a time-stack and applies scale_bar and timestamp to every frame.
def annotate(
    rgb_stack: np.ndarray,
    xPixel_size_um: float,
    deltaTime: float,
    Add_Scalebar: bool,
    Add_Timestamp: bool,
    IsTimelapse: bool
):
    annotated_frames = []

    for t in range(rgb_stack.shape[0]):
        frame = rgb_stack[t].copy()  # IMPORTANT: avoid in-place modification

        if Add_Scalebar:
            frame, scale_bar_len_um = scale_bar(frame, xPixel_size_um)
            
        if Add_Timestamp and IsTimelapse:
            frame = timestamp(frame, t, deltaTime) # Here t is the frame index

        annotated_frames.append(frame)
    print(f"Adding Scale bar of {scale_bar_len_um:g} µm with label.")
    print(f"Adding Timestamp.")

    return np.stack(annotated_frames, axis=0)

# -----------------------------------------------------------------------------
# 2.4 VIDEO SAVING FUNCTION
# -----------------------------------------------------------------------------

# Function to pad frames to be multiples of block size
# Logic: Pads the image stack with zeros to ensure width and height are multiples of 'block' (e.g., 16 for H.264).
def framesize_pad(
    img: np.ndarray,
    order: str,
    block: int = 16 #Imageio ffmeg_writer: macro_block_size=16, i.e., width and height must be multiples of 16
):
    # Identify Y and X axes from dimension order
    y_axis = order.index('Y')
    x_axis = order.index('X')

    h = img.shape[y_axis]
    w = img.shape[x_axis]

    # Compute padded sizes (round up to nearest multiple of block)
    new_h = ((h + block - 1) // block) * block
    new_w = ((w + block - 1) // block) * block

    pad_h = new_h - h
    pad_w = new_w - w

    # Initialize pad_width with no padding on all axes
    pad_width = [(0, 0)] * img.ndim

    # Apply padding only on Y and X axes
    pad_width[y_axis] = (0, pad_h)
    pad_width[x_axis] = (0, pad_w)

    if pad_h > 0 or pad_w > 0:
        print(
            f"Padding adjustment for video encoding: "
            f"Original Size=({w}x{h}), "
            f"Padded Size=({new_w}x{new_h})"
        )
    # Apply padding
    padded = np.pad(img, pad_width, mode="constant", constant_values=0)
    return padded

# Function to export MP4 video
# Logic: Uses bioio_imageio to save the numpy stack as an MP4 file.
def MP4_Export(
    rgb_stack: np.ndarray,
    output_dir: str | Path,
    file_name: str,
    scene_idx: int,
    channel_idx: int | None,
    fps: int,
    verbose=True
):

    # Naming convention based on channel index
    if channel_idx is None:
        channel_tag = "C0"  # merged
        label = "Merged"
    else:
        channel_tag = f"C{channel_idx + 1}" # e.g., Index 0 becomes C1
        label = f"Channel {channel_idx + 1}"

    output_video_path = (Path(output_dir) / f"{file_name}_S{scene_idx + 1}_{channel_tag}.mp4")
    
    # Save video using ffmpeg wrapper
    TimeseriesWriter.save(
        rgb_stack,
        output_video_path,
        fps=fps,
        codec="libx264",
        quality=10
    )

    if verbose:
        print(
            f"Scene {scene_idx + 1}, {label} video saved at: "
            f"{output_video_path}")

### 2. User Configuration

In [ ]:
# Input Directory
WorkingDir = Path(r"D:\02_VSI Working Directory\UnderProcessing\20250714_D161-iMyoD-PB24-D5_BFP-ftRLC-LfAct-6SWT_Lv-01Hr") # CHANGE THIS PATH TO YOUR DATA DIRECTORY

# User configurations
FileToProcess = "all"               # "all" or filename
ScenesToProcess = "all"             # "all" or specific list [0, 1, 2]
Proj_mode = "max"                      # max, in mean, sum, min, sd (Projection method)
Proj_axis = "Z"                   # Axis to project (z or T)
FPS = 5                            # output video frames per second
Add_Scalebar = True                 # whether to add scalebar
Add_Timestamp = True                # whether to add timestamp

# Channel information
# Channel index (0–3), matching the acquisition order in the imaging file
# Maps index to Name and Color for visualization
ChannelInfo = { 
    0: {"Name": "NLS-BFP",   "cmap": "yellow"},
    1: {"Name": "ftRLC-mSG", "cmap": "magenta"},
    2: {"Name": "LfAct-mSc3",    "cmap": "green"},
    3: {"Name": "m6SWT-Halo7",      "cmap": "cyan"},
}

# For Debugging
DryRun = False                       # if True, only the first scene of the first image will be processed.

### 3. Main Execution Block

In [ ]:
# Create output directory
OutputDir = MakeDir(WorkingDir, Suffix="Video") 
print("Output Directory:", OutputDir)
# Get list of files to process based on config
FilesToProcess = FileList(WorkingDir, FileToProcess, DryRun) 

# --- LOOP THROUGH FILES ---
for file in FilesToProcess: 
    print("Processing file:", file)

    FileName = rm_ext(file.name) # Remove file extension
    print("File name without extension:", FileName)

    # Make sub-directory for each file
    FileOutputDir = MakeSubDir(OutputDir, FileName)

    img = BioImage(file, reader=bioio_nd2.Reader)   # Load ND2 image using BioImage

    # Extract and print metadata
    MD = ND2_Metadata(img)

    # --- LOOP THROUGH SCENES (Positions) ---
    for s in SceneList(img, ScenesToProcess, DryRun):  # Get list of scenes to process
        print(f"Processing scene: {int(s)+1}")
        
        # Set scene (lazy loading specific scene)
        img.set_scene(s)

        cimg_list = []  # List to store individual processed channels for later merging

        # --- LOOP THROUGH CHANNELS ---
        for j in range(MD["nChannels"]):
            # Get image data for the current channel
            cimg, order = get_channel_data(img, MD, channel_idx=j)

            cimg = framesize_pad(cimg, order)  # Padding frames to be multiples of block size for video encoding
            
            print(f"Processing Channel {j+1}/{MD['nChannels']}")
            print(f"{order} with shape: {cimg.shape}")

            # Determine if the dataset is a timelapse
            IsTimelapse_flag = IsTimelapse(MD, order)

            if cimg.ndim > 3:
                # Performing Projection
                cimg_proj = project(
                    stack=cimg,
                    mode=Proj_mode,
                    axis=Proj_axis,
                    dim_order=order,
                    keepdims=False
                )
            else:
                print(f"Skipping projection due to unsupported dimensionality: {cimg.shape}")
                cimg_proj = cimg  # No projection applied

            print(f"Projected shape: {cimg_proj.shape}")

            # Percentile normalization (scales intensities 0.0 to 1.0)
            cimg_mip_norm = norm_img(cimg_proj, percent_norm=True)

            # Applying cmap (LUT) for visualization -> Converts to RGB
            cmap_cimg_mip_norm = cmap(img=cimg_mip_norm, LUT=ChannelInfo[j]["cmap"], Isinverted=False)

            print(f"Colormapped shape: {cmap_cimg_mip_norm.shape}")

            # Annotating frames with scalebar and timestamp
            cimg_mip_rgb_labeled = annotate(
                rgb_stack=cmap_cimg_mip_norm,
                xPixel_size_um=MD["xPixel_size_um"],
                deltaTime=MD["deltaTime"],
                Add_Scalebar=Add_Scalebar,
                Add_Timestamp=Add_Timestamp,
                IsTimelapse=IsTimelapse_flag)

            # Save individual channel as MP4
            MP4_Export(
                rgb_stack=cimg_mip_rgb_labeled,
                output_dir=FileOutputDir,
                file_name=FileName,
                scene_idx=int(s),
                channel_idx=j,
                fps=FPS,
                verbose=True)

            # Updating the cimg_list for the merged view
            cimg_list.append(cmap_cimg_mip_norm.astype(np.float32)) # Converting to float32 for accurate summation in merge channel

        # --- MERGE CHANNELS ---
        # Summing channel intensities and clipping to valid 8-bit range (0-255)
        merged_rgb = np.clip(sum(cimg_list), 0, 255).astype(np.uint8) # shape: [T, C, Y, X]

        print(f"Merged RGB shape: {merged_rgb.shape}")

        # Adding scale bar/timestamp to the Merged Stack
        merged_mip_rgb_8bit_labeled = annotate(
            rgb_stack=merged_rgb,
            xPixel_size_um=MD["xPixel_size_um"],
            deltaTime=MD["deltaTime"],
            Add_Scalebar=Add_Scalebar,
            Add_Timestamp=Add_Timestamp,
            IsTimelapse=IsTimelapse_flag)

        # Save Merged Stack as MP4 (Channel index None indicates 'Merged')
        MP4_Export(
            rgb_stack=merged_mip_rgb_8bit_labeled,
            output_dir=FileOutputDir,
            file_name=FileName,
            scene_idx=int(s),
            channel_idx=None,  
            fps=FPS,
            verbose=True)

### 4. Appendix - Renaming Script

In [ ]:
# Renaming the videos for ppt presentation
# Logic: Generates sequential names (Video-1, Video-2...) to preserve order in slides/folder views.

nChannels = int(MD["nChannels"])

print("OutputDir:", OutputDir)
OutputDir_Renamed = MakeDir(WorkingDir, Suffix="Video-Renamed")

# Iterate through each subfolder inside OutputDir
for subdir in sorted([d for d in OutputDir.iterdir() if d.is_dir()]):
    print(f"\nProcessing folder: {subdir.name}")

    OutputSubDir = MakeSubDir(OutputDir_Renamed, subdir.name)

    FilePrefix = subdir.name

    nFiles = sum(1 for f in subdir.iterdir() if f.is_file())
    print(f"Number of files in {subdir.name}: {nFiles}")
    nSeries = nFiles // (nChannels + 1)  # +1 for merged channel

    counter = 1  # reset counter per subfolder

    if DryRun:
        print("Dry run enabled. No files will be renamed.")
        continue

    # Iterate through Scenes (1 to N)
    for s in range(1, nSeries + 1):
        # Iterate backwards through channels + merged
        for c in range(nChannels, -1, -1):
            old_filename = f"{FilePrefix}_S{s}_C{c}.mp4"
            new_filename = f"Video-{counter}.mp4"

            old_filepath = subdir / old_filename
            new_filepath = Path(OutputSubDir) / new_filename

            if not old_filepath.exists():
                print(f"  Skipping missing file: {old_filepath.name}")
                continue

            shutil.copy2(old_filepath, new_filepath)
            counter += 1

    print("Renamed videos saved in:", OutputDir_Renamed)